In [2]:
pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/9a/b6/2e2a011b2dc27a6711376808b4cd8c922c476ea0f1420b39892117fa8563/openai-1.61.1-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/f5/97/0468bc9eeae43079aaa5feb9267964e496bf13133d469cfdc135498f8dd0/jiter-0.8.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/87/f5/72347bc88306acb359581ac4d52f23c0ef445b57157adedb9aee0cd689d2/h

In [1]:
import requests
import json
import re

# LM Studio의 로컬 서버 URL
LM_STUDIO_API_URL = "http://localhost:1234/v1"

# 사용할 모델 이름 설정 (LM Studio에서 활성화한 모델과 일치해야 함)
MODEL_NAME = "llama-3.2-1b-instruct"
# MODEL_NAME = "deepseek-r1-distill-qwen-1.5b"

# GET 요청 예제 (서버에 로드된 모델 확인)
def get_models():
    response = requests.get(f"{LM_STUDIO_API_URL}/models")
    return response.json()

# 프롬프트 엔지니어링 적용
def generate_prompt(user_input):
    prompt_template = f"""
### INSTRUCTION ###
Extract structured information from the given user instruction and format the output as JSON.

### JSON Output Format ###
{{
    "action": {{
        "type": "drag" | "click" | "slide",   
        "target": {{
            "type": "axis" | "visual element" | "button" | "slider",
            "label": "x-axis" | "y-axis" | "area chart" | "sort" | "filter"
        }},
        "destination": "horizontal" | "vertical" | "x-axis" | "y-axis" | "visual element" | "outside canvas" (optional)
    }},
    "result": {{
        "type": "sort" | "swap" | "zoom" | "filter" | "highlight",    
        "target": "visual element" | "x-axis" | "y-axis" | "axes" | "area chart" | "stacked bar chart" | "grouped bar chart" | "bubble chart" | "line chart" | "bar chart",  
        "parameter": "color" | "position" | "height" | "order" | "range"
    }}
}}

### RULES ###
1. Response **MUST** be in JSON format without any additional explanation.
2. **Do not** include any extra text, only return the **single** JSON object.
3. `"destination"` field **MUST ONLY** be included for `"drag"` actions.
4. **Directional Mapping:**  
   - If the input contains `"left and right"`, set `"destination": "horizontal"`.  
   - If the input contains `"up and down"`, set `"destination": "vertical"`.  
5. **Slider Rules:**  
   - If `"slider"`, `"slide"`, or `"filter"` appears in the input, set `"action.type": "slide"`.  
   - `"target.type"` **MUST** be `"slider"`, and `"target.label"` **MUST** be `"filter"`.  
   - `"destination"` field **MUST NOT** be present for `"slide"` actions.  
6. Ensure all values match the predefined options exactly.


### EXAMPLES ###
#### Example 1
INPUT:
"Create a slider with five steps (1 to 5) that highlights visual elements within the selected range."  

OUTPUT:
{{
    "action": {{
        "type": "slide",
        "target": {{
            "type": "slider",
            "label": "filter"
        }}
    }},
    "result": {{
        "type": "highlight",
        "target": "visual elements",
        "parameter": "range"
    }}
}}

#### Example 2
INPUT:
"Allow drag the x-axis to the y-axis to swap the position of the axis."

OUTPUT:
{{
    "action": {{
        "type": "drag",
        "target": {{
            "type": "axis",
            "label": "x-axis"
        }},
        "destination": "y-axis"
    }},
    "result": {{
        "type": "swap",
        "target": "axes",
        "parameter": "position"
    }}
}}

#### Example 3
INPUT:
"Allow users to drag the x-axis left and right to zoom in on the x-axis range."

OUTPUT:
{{
    "action": {{
        "type": "drag",
        "target": {{
            "type": "axis",
            "label": "x-axis"
        }},
        "destination": "horizontal"
    }},
    "result": {{
        "type": "zoom",
        "target": "x-axis",
        "parameter": "range"
    }}
}}


### INPUT ###
"{user_input}"

### RESPONSE (Only ONE JSON, no explanation) ###
"""
    return prompt_template.strip()

def chat_with_model(prompt):
    headers = {"Content-Type": "application/json"}
    data = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant. Always return only JSON."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.0  # 낮은 온도로 정확한 응답 유도
    }
    response = requests.post(f"{LM_STUDIO_API_URL}/chat/completions", json=data, headers=headers)
    return response.json()

def extract_json(response_text):
    try:
        # JSON 객체만 찾기 위해 정규식을 사용하지 않고 직접 변환
        return json.loads(response_text.strip())
    except json.JSONDecodeError:
        return {"error": "Invalid JSON format"}

def main():
    # GET 요청 실행
    models = get_models()
    print("Available Models:", models)
    
    while True:
        # 사용자 입력 받기
        user_input = input("User: ")
        
        # 종료 명령어 처리
        if user_input.lower() in ["exit", "quit", "q"]:
            print("Exiting program...")
            break
        
        print("\n### Input ###")
        print(user_input)
    
        structured_prompt = generate_prompt(user_input)
        chat_response = chat_with_model(structured_prompt)
        response_text = chat_response["choices"][0]["message"]["content"]
        
        # print("\n### Response Text ###")
        # print(response_text)

        extracted_json = extract_json(response_text)
        if extracted_json == {"error": "Invalid JSON format"}:
            print("\n### Output ###")
            print(response_text)
        else:    
            print("\n### Structured Output ###")
            print(json.dumps(extracted_json, indent=4, ensure_ascii=False))
        
if __name__ == "__main__":
    main()

Available Models: {'data': [{'id': 'llama-3.2-1b-instruct', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'deepseek-r1-distill-qwen-1.5b', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'text-embedding-nomic-embed-text-v1.5', 'object': 'model', 'owned_by': 'organization_owner'}], 'object': 'list'}
Exiting program...


In [2]:
!pip install ipykernel notebook
!pip install flask
!pip install ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 9.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 7.8 MB/s eta 0:00:00 0:00:01m


In [2]:
from flask import Flask, request, jsonify
import requests
import json

app = Flask(__name__)


# LM Studio의 로컬 서버 URL
LM_STUDIO_API_URL = "http://localhost:1234/v1"

# 사용할 모델 이름 설정 (LM Studio에서 활성화한 모델과 일치해야 함)
MODEL_NAME = "llama-3.2-1b-instruct"
# MODEL_NAME = "deepseek-r1-distill-qwen-1.5b"

# GET 요청 예제 (서버에 로드된 모델 확인)
def get_models():
    response = requests.get(f"{LM_STUDIO_API_URL}/models")
    return response.json()

# 프롬프트 엔지니어링 적용
def generate_prompt(user_input):
    prompt_template = f"""
### INSTRUCTION ###
Extract structured information from the given user instruction and format the output as JSON.

### JSON Output Format ###
{{
    "action": {{
        "type": "drag" | "click" | "slide",   
        "target": {{
            "type": "axis" | "visual element" | "button" | "slider",
            "label": "x-axis" | "y-axis" | "area chart" | "sort" | "filter"
        }},
        "destination": "horizontal" | "vertical" | "x-axis" | "y-axis" | "visual element" | "outside canvas" (optional)
    }},
    "result": {{
        "type": "sort" | "swap" | "zoom" | "filter" | "highlight",    
        "target": "visual element" | "x-axis" | "y-axis" | "axes" | "area chart" | "stacked bar chart" | "grouped bar chart" | "bubble chart" | "line chart" | "bar chart",  
        "parameter": "color" | "position" | "height" | "order" | "range"
    }}
}}

### RULES ###
1. Response **MUST** be in JSON format without any additional explanation.
2. **Do not** include any extra text, only return the **single** JSON object.
3. `"destination"` field **MUST ONLY** be included for `"drag"` actions.
4. **Directional Mapping:**  
   - If the input contains `"left and right"`, set `"destination": "horizontal"`.  
   - If the input contains `"up and down"`, set `"destination": "vertical"`.  
5. **Slider Rules:**  
   - If `"slider"`, `"slide"`, or `"filter"` appears in the input, set `"action.type": "slide"`.  
   - `"target.type"` **MUST** be `"slider"`, and `"target.label"` **MUST** be `"filter"`.  
   - `"destination"` field **MUST NOT** be present for `"slide"` actions.  
6. Ensure all values match the predefined options exactly.


### EXAMPLES ###
#### Example 1
INPUT:
"Create a slider with five steps (1 to 5) that highlights visual elements within the selected range."  

OUTPUT:
{{
    "action": {{
        "type": "slide",
        "target": {{
            "type": "slider",
            "label": "filter"
        }}
    }},
    "result": {{
        "type": "highlight",
        "target": "visual elements",
        "parameter": "range"
    }}
}}

#### Example 2
INPUT:
"Allow drag the x-axis to the y-axis to swap the position of the axis."

OUTPUT:
{{
    "action": {{
        "type": "drag",
        "target": {{
            "type": "axis",
            "label": "x-axis"
        }},
        "destination": "y-axis"
    }},
    "result": {{
        "type": "swap",
        "target": "axes",
        "parameter": "position"
    }}
}}

#### Example 3
INPUT:
"Allow users to drag the x-axis left and right to zoom in on the x-axis range."

OUTPUT:
{{
    "action": {{
        "type": "drag",
        "target": {{
            "type": "axis",
            "label": "x-axis"
        }},
        "destination": "horizontal"
    }},
    "result": {{
        "type": "zoom",
        "target": "x-axis",
        "parameter": "range"
    }}
}}


### INPUT ###
"{user_input}"

### RESPONSE (Only ONE JSON, no explanation) ###
"""
    return prompt_template.strip()

def chat_with_model(prompt):
    headers = {"Content-Type": "application/json"}
    data = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant. Always return only JSON."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.0  # 낮은 온도로 정확한 응답 유도
    }
    response = requests.post(f"{LM_STUDIO_API_URL}/chat/completions", json=data, headers=headers)
    return response.json()

def extract_json(response_text):
    try:
        # JSON 객체만 찾기 위해 정규식을 사용하지 않고 직접 변환
        return json.loads(response_text.strip())
    except json.JSONDecodeError:
        return {"error": "Invalid JSON format"}

@app.route('/speech_to_json', methods=['POST'])
def speech_to_json():
    data = request.get_json()
    if 'text' not in data:
        return jsonify({"error": "Missing 'text' field in request body"}), 400
    
    user_input = data['text']
    structured_prompt = generate_prompt(user_input)
    chat_response = chat_with_model(structured_prompt)
    response_text = chat_response["choices"][0]["message"]["content"]
    extracted_json = extract_json(response_text)
    
    return jsonify(extracted_json)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.91.217.18:5001
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/leejaeuk/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/leejaeuk/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/leejaeuk/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/leejaeuk/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/Users/leejaeuk/anaconda3/lib/python3.11/site-packages/ip

SystemExit: 1